### Data Cleaning

In [ ]:
##import das bibliotecas e adequando colunas, linhas e formato de números

from google.cloud import bigquery
from dotenv import load_dotenv
load_dotenv()

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import re

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json"


# Configurar Pandas para exibir todas as colunas e todas as linhas completas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)


pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# Configurar o cliente do BigQuery
client = bigquery.Client()

# Nome do dataset e tabela
dataset_id = 'raw_data'

In [ ]:
# Listar tabelas no dataset
tables = client.list_tables('raw_data')
print("Tabelas disponíveis:")
for table in tables:
    print(table.table_id)




In [ ]:
query = f"SELECT * FROM `raw_data.humanresources_employee`"
data = client.query(query).result().to_dataframe()

# Expandir a coluna JSON
raw_data = pd.json_normalize(data['data'])

# Exibir os dados expandidos
print(raw_data.sample(n=50))


### Verificando valores nulos/em branco nos dados

In [ ]:
#valores nulos

raw_data.isnull().sum()

In [ ]:
#total valores unicos de cada variável

valores_unicos = []

for i in raw_data.columns[0:15].tolist():
    print(i, ':', len(raw_data[i].astype(str).value_counts()))
    valores_unicos.append(len(raw_data[i].astype(str).value_counts()))

In [ ]:
# Identificar duplicatas com base em 'businessentityid'
duplicatas = raw_data[raw_data.duplicated(subset=['businessentityid'], keep=False)]

# Verificar se existem duplicatas
if not duplicatas.empty:
    # Ordenar duplicatas por 'businessentityid' e 'modifieddate'
    duplicatas_ordenadas = duplicatas.sort_values(by=['businessentityid', 'modifieddate'])
    
    # Exibir duplicatas ordenadas
    print("Duplicatas ordenadas:")
    print(duplicatas_ordenadas)
else:
    print("Não foram encontradas duplicatas.")


In [ ]:
print(duplicatas_ordenadas.drop_duplicates())


In [ ]:
duplicados_businessentityid = raw_data[raw_data.duplicated(subset=['businessentityid'], keep=False)]

# Ordenar por 'businessentityid' para facilitar a análise
duplicados_ordenados = duplicados_businessentityid.sort_values(by=['businessentityid'])

# Exibir todas as linhas duplicadas
print(duplicados_ordenados)


In [ ]:
pd.set_option('display.max_rows', None)
contagem = raw_data['businessentityid'].value_counts()

# Filtrar apenas os IDs que aparecem mais de uma vez
repetidos = contagem[contagem > 1]

# Exibir repetidos novamente
print(repetidos)

In [ ]:
#copia da humanresources_employee
raw_data_bkp = raw_data.copy()

# Ordenar o DataFrame por 'businessentityid' e 'modifieddate'
raw_data = raw_data.sort_values(by=['businessentityid', 'modifieddate'])

# Remover duplicatas mantendo a última ocorrência com base em 'modifieddate'
raw_data = raw_data.drop_duplicates(subset=['businessentityid'], keep='last')

print(f"Linhas após remover duplicatas (baseando-se na última 'modifieddate'): {len(raw_data)}")


In [ ]:
# Verificar informações do DataFrame
print(raw_data.info())


In [ ]:
# Identificar colunas com datas
date_columns = ['birthdate', 'hiredate', 'modifieddate']

# Converter as colunas para datetime se ainda não estiverem
for col in date_columns:
    raw_data[col] = pd.to_datetime(raw_data[col], errors='coerce')

# Criar uma cópia do DataFrame para exibição formatada
formatted_data = raw_data.copy()

# Formatar todas as colunas de datas para exibição
for col in date_columns:
    formatted_data[col] = raw_data[col].dt.strftime('%Y-%m-%d %H:%M:%S')

# Exibir o DataFrame formatado
print(formatted_data.head())

# Verificar os tipos originais permanecem datetime64[ns]
print("\nTipos originais das colunas no DataFrame principal:")
print(raw_data[date_columns].dtypes)


In [ ]:
# Padronizar textos em jobtitle e gender
raw_data['jobtitle'] = raw_data['jobtitle'].str.strip().str.title()
raw_data['gender'] = raw_data['gender'].str.strip().str.upper()

# Verificar valores únicos para garantir a padronização
print("Valores únicos em 'jobtitle':", raw_data['jobtitle'].unique())
print("Valores únicos em 'gender':", raw_data['gender'].unique())


In [ ]:
# Identificar colunas numéricas para análise 
numeric_columns = ['sickleavehours', 'vacationhours']

# Exibir estatísticas descritivas
print(raw_data[numeric_columns].describe())

# Calcular limites para outliers (IQR - Intervalo Interquartil)
for col in numeric_columns:
    q1 = raw_data[col].quantile(0.25)
    q3 = raw_data[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Exibir os limites
    print(f"\nColuna: {col}")
    print(f"Limite inferior: {lower_bound}, Limite superior: {upper_bound}")
    
    # Filtrar outliers
    outliers = raw_data[(raw_data[col] < lower_bound) | (raw_data[col] > upper_bound)]
    print(f"Outliers detectados ({len(outliers)}):")
    print(outliers[[col]])


In [ ]:

# Criar a coluna 'hire_year' com base em 'hiredate'
raw_data['hire_year'] = raw_data['hiredate'].dt.year


#verificando outros dados para detectar outliers
anos_contratacao = [int(ano) for ano in raw_data['hire_year'].unique()]
anos_contratacao.sort()
print(anos_contratacao)

print("Valores únicos em 'gender':", raw_data['gender'].unique())
print("Valores únicos em 'salariedflag':", raw_data['salariedflag'].unique())


In [ ]:
# Verificar valores ausentes
missing = raw_data.isnull().sum()
print("Valores ausentes por coluna:")
print(missing)

# Tratar colunas críticas
if missing['hiredate'] > 0:
    print("Tratar valores ausentes em 'hiredate' (decisão: remover ou imputar)")

if missing['jobtitle'] > 0:
    print("Tratar valores ausentes em 'jobtitle' (decisão: remover ou imputar)")

if missing['businessentityid'] > 0:
    print("Erro crítico: 'businessentityid' não pode ter valores ausentes!")


In [ ]:
# Verificar linhas com 'modifieddate' ausente
missing_modifieddate = raw_data[raw_data['modifieddate'].isnull()]
print("Linhas com 'modifieddate' ausente:")
print(missing_modifieddate)


In [ ]:
# Preencher 'modifieddate' ausente ou igual a 'hiredate', pois pode ser a ultima data de modificação no sistema.
raw_data.loc[raw_data['modifieddate'].isnull() | (raw_data['modifieddate'] == pd.Timestamp('1900-01-01')), 'modifieddate'] = raw_data['hiredate']

# Exibir as linhas ajustadas
print("Linhas onde 'modifieddate' foi ajustado para 'hiredate':")
print(raw_data.loc[raw_data['modifieddate'] == raw_data['hiredate']])


In [ ]:
# Verificar unicidade de 'businessentityid'
is_unique = raw_data['businessentityid'].is_unique
print(f"'businessentityid' é único? {is_unique}")


In [ ]:
# Definir regex para validar números (exemplo: apenas dígitos, 9 caracteres)
regex = r'^\d{9}$'

# Verificar valores inválidos
invalid_nationalid = raw_data[~raw_data['nationalidnumber'].astype(str).str.match(regex)]
print(f"Valores inválidos em 'nationalidnumber':\n{invalid_nationalid['nationalidnumber']}")


In [ ]:
regex_email = r'^[\w\.-]+@[\w\.-]+\.\w+$'
invalid_loginid = raw_data[~raw_data['loginid'].str.match(regex_email)]
print(f"Valores inválidos em 'loginid':\n{invalid_loginid['loginid']}")


In [ ]:
is_unique = raw_data['nationalidnumber'].is_unique
print(f"'nationalidnumber' é único? {is_unique}")


In [ ]:
# Criar um backup do DataFrame tratado
raw_data_bkp_v2 = raw_data.copy()

# Verificar o tamanho do backup e as primeiras linhas
print(f"Backup criado com {len(raw_data_bkp_v2)} linhas.")
print(raw_data_bkp_v2.head())


In [ ]:
# Verificar e documentar colunas existentes
print("Colunas mantidas no dataset:", raw_data.columns.tolist())


In [ ]:
for col in raw_data.columns:
    print(f"Valores únicos em '{col}':", raw_data[col].unique()[:10])  # Limitar a exibição a 10 valores



In [ ]:
# Listar colunas binárias esperadas
binary_columns = ['currentflag', 'salariedflag']

# Verificar valores únicos em colunas binárias
for col in binary_columns:
    unique_values = raw_data[col].unique()
    print(f"Valores únicos em '{col}': {unique_values}")

    # Corrigir valores não binários, se necessário
    if not set(unique_values).issubset({True, False, 0, 1}):
        pr



In [ ]:
# Contar valores em 'currentflag' e 'salariedflag'
print("Distribuição de 'currentflag':")
print(raw_data['currentflag'].value_counts())

print("\nDistribuição de 'salariedflag':")
print(raw_data['salariedflag'].value_counts())


In [ ]:
# 1. Verificar se todos os funcionários ativos têm currentflag = True, pois deveria ser false = demitido/desligado
print("Funcionários ativos errados:", raw_data[raw_data['currentflag'] != True])

# 2. Validar datas
print("Contratações futuras:", raw_data[raw_data['hiredate'] > pd.Timestamp.now()])
print("Modifieddate antes de hiredate:", raw_data[raw_data['modifieddate'] < raw_data['hiredate']])

# 3. Verificar unicidade de identificadores
print("Duplicados em 'businessentityid':", raw_data['businessentityid'].duplicated().sum())
print("Duplicados em 'nationalidnumber':", raw_data['nationalidnumber'].duplicated().sum())



## ESTATÍSTICA DESCRITIVA

In [ ]:
# Selecionar colunas relevantes para análise descritiva
cols_para_analise = ['sickleavehours', 'vacationhours', 'salariedflag']

# Garantir que as datas estejam no formato correto
raw_data['hire_year'] = pd.to_datetime(raw_data['hiredate']).dt.year

# Adicionar a nova coluna à lista
cols_para_analise.append('hire_year')

# Gerar estatísticas descritivas
analise_descritiva = raw_data[cols_para_analise].describe(include='all')

# Substituir NaN por '-'
analise_descritiva = analise_descritiva.fillna('-')

print(analise_descritiva)
